In [1]:
# tour.interpark.com 에서 여행 정보 검색(예:로마)해서 원하는 정보 가져오기

In [2]:
try:
    from selenium import webdriver
except:
    ! pip install selenium

In [3]:
try:
    from webdriver_manager.chrome import ChromeDriverManager
except:
    ! pip install webdriver-manager

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from bs4 import BeautifulSoup

In [5]:
main_url = 'https://tour.interpark.com/'
keyword = '로마'
tour_list = []
delay = 5 # 5초

In [6]:
driver = webdriver.Chrome( service=ChromeService( ChromeDriverManager().install() ) ) 

In [7]:
driver.get(main_url)
sleep(delay)

In [8]:
# <input id='SearchGNBText'...>
try:
    search_box = WebDriverWait(driver, delay).until(
        EC.presence_of_element_located( (By.CSS_SELECTOR, '#SearchGNBText') ) )
except TimeoutException:
    print("로딩 시간 초과!")
search_box.send_keys(keyword)

In [9]:
# <button class='search-btn'...>
driver.find_element(By.CSS_SELECTOR, 'button.search-btn').click()
sleep(delay)

In [10]:
# <div class='searchAllBox overseaTravel col1'> ... <button class='moreBtn'...>
more_btn = driver.find_element(By.CSS_SELECTOR, '.searchAllBox.overseaTravel.col1 .moreBtn')
print(more_btn)
print(more_btn.text)
more_btn.click()
sleep(delay)

<selenium.webdriver.remote.webelement.WebElement (session="3a190aaa4dd3b118ff0399ad18e465de", element="880cbde5-a760-4d51-9dce-0202111ff908")>
해외여행 더보기


In [11]:
# <button class='nextAllBtn'...>
driver.find_element(By.CSS_SELECTOR, '.nextAllBtn').click()
sleep(delay)

In [12]:
# <div class='pageNumBox'..> <ul> ... <li>...
str_last_pages = driver.find_elements(By.CSS_SELECTOR, '.pageNumBox li')
str_last_page = str_last_pages[-1].text # 마지막의 글자
last_page = int( str_last_page )
print(last_page) # 5 (숫자)

5


In [13]:
sleep(1)
# <button class='prevAllBtn'...>
driver.find_element(By.CSS_SELECTOR, '.prevAllBtn').click()
sleep(delay)

In [14]:
from tqdm.notebook import tqdm

In [15]:
sleep(1)
# 1페이지
links = []
# <div class='searchAllBox overseaTravel col1'> <li class='boxItem'>
boxItems = driver.find_elements(By.CSS_SELECTOR,'.searchAllBox.overseaTravel.col1 .boxItem')
# len(boxItems) # 링크 포함한 boxItem 10개
for boxItem in tqdm( boxItems ):
    links.append( boxItem.get_attribute('data-id') )
links

  0%|          | 0/10 [00:00<?, ?it/s]

['D3010721',
 'A3018602',
 'A3015008',
 'A3018067',
 'D3010725',
 'D3010724',
 'D3010714',
 'D3010713',
 'A3018106',
 'A3018099']

In [16]:
sleep(1)
# 2부터 5페이지 : 총 4번 클릭
for page_link in tqdm( range(1, last_page) ): # 1,2,3,4(last_page-1) ==> 4번 클릭
    driver.find_element(By.CSS_SELECTOR, '.pageNumBox .nextBtn').click()
    sleep(delay)
    boxItems = driver.find_elements(By.CSS_SELECTOR,
                                    '.searchAllBox.overseaTravel.col1 .boxItem')
    for boxItem in tqdm( boxItems ):
        links.append(boxItem.get_attribute('data-id')) # <li class="boxItem" data-id="D3010721"

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
sleep(1)
links

['D3010721',
 'A3018602',
 'A3015008',
 'A3018067',
 'D3010725',
 'D3010724',
 'D3010714',
 'D3010713',
 'A3018106',
 'A3018099',
 'A3017982',
 'A3018207',
 'A3018290',
 'A3012938',
 'A3017954',
 'A3014578',
 'A3014604',
 'A3012471',
 'A3014154',
 'A3018534',
 'Q2010037',
 'A3018479',
 'A3018362',
 'A3018284',
 'A3017907',
 'A3017103',
 'A3018600',
 'A3018598',
 'A3018580',
 'A3018578',
 'A3018563',
 'A3018531',
 'A3018525',
 'A3018408',
 'A3018391',
 'A3017997',
 'A3017956',
 'A3017828',
 'A3017748',
 'A3017745',
 'A3017685',
 'A3016731',
 'A1033117']

In [18]:
# [유럽허니문] 스위스+이탈리아 8박11일 (인터라켄2박+베니스1박+피렌체2박+로마3박)
# http://tour.interpark.com/goods/detail/?BaseGoodsCd=D3010721

In [19]:
sleep(1)
base_url = "http://tour.interpark.com/goods/detail/?BaseGoodsCd="
# search_links = [ base_url+link for link in links ]
search_links = [ base_url+link for link in links[:2] ] # 2개의 링크만 테스트
search_links

['http://tour.interpark.com/goods/detail/?BaseGoodsCd=D3010721',
 'http://tour.interpark.com/goods/detail/?BaseGoodsCd=A3018602']

In [20]:
sleep(1)
tour_datas = []
for link in tqdm( search_links ):
    print("link=", link)
    driver.get(link)
    sleep(delay)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    # <div class="default-section goods-info"> <h2> <strong>상품명
    title = soup.select_one('.default-section.goods-info h2 strong').string # 상품명
    tbody = soup.select_one('.ui-data-table.min tbody') # <tbody> 태그 얻기
    code = tbody.tr.td.strong.string # <tbody><tr><td><strong>22091412409</strong>,상품코드
    td1 = tbody.select('td')[1] # 인덱스로 1번째(사람기준 2번째)
    td1.strong.div.decompose() # <strong> 하위의 <div> 태그를 없앤다.<strong>XXX가 필요해서
    period = td1.select_one('strong').get_text(strip=True) # 공백 제거, 여행기간
    td2 = tbody.select('td')[2] # 인덱스로 2번째(사람기준 2번째)
    reserved = td2.strong.string
    totalPay = soup.select_one('.j-totalPay').string
    d_title = {"상품명":title}
    d_code = {"상품코드":code}
    d_period = {"여행기간":period}
    d_reserved = {"예약현황":reserved}
    d_totalPay = {"총 합계 금액":totalPay}
    tour_datas.append([ d_title, d_code, d_reserved, d_totalPay ])
tour_datas

  0%|          | 0/2 [00:00<?, ?it/s]

link= http://tour.interpark.com/goods/detail/?BaseGoodsCd=D3010721
link= http://tour.interpark.com/goods/detail/?BaseGoodsCd=A3018602


[[{'상품명': '[유럽허니문] 스위스+이탈리아 8박11일 (인터라켄2박+베니스1박+피렌체2박+로마3박)'},
  {'상품코드': '22091412409'},
  {'예약현황': '예약 0명'},
  {'총 합계 금액': '3,290,000'}],
 [{'상품명': '[주말출도착 이탈리아 9일]2대야경(로마/피렌체)+포토스팟피사_터키항공'},
  {'상품코드': '22110211484'},
  {'예약현황': '예약 5명'},
  {'총 합계 금액': '1,841,400'}]]

In [21]:
driver.close()
driver.quit()

In [22]:
# end